Adaptado de [Machine Learning in Production](https://www.deeplearning.ai/courses/machine-learning-in-production/) de [Andrew Ng](https://www.deeplearning.ai/)  ([Stanford University](http://online.stanford.edu/), [DeepLearning.AI](https://www.deeplearning.ai/))

# Servir um modelo com o TensorFlow Serving
------------------------

Neste laboratório, você dará uma olhada no sistema de fornecimento de modelos da TFX para produção chamado [Tensorflow Serving](https://www.tensorflow.org/tfx/guide/serving). Esse sistema é altamente integrado ao Tensorflow e oferece uma maneira fácil e direta de implantar modelos.

Especificamente, você irá:
- Aprender a instalar o TF Serving.
- Carregar um modelo pré-treinado que classifica cães, pássaros e gatos.
- Salve-o seguindo as convenções necessárias para o TF serving.
- Rodar um servidor Web usando o TF serving que aceitará solicitações por HTTP.
- Ingeragir com seu modelo por meio de uma API REST.
- Saiba mais sobre o controle de versão do modelo.

Este laboratório se inspira [nesse tutorial oficial do Tensorflow](https://www.tensorflow.org/tfx/tutorials/serving/rest_simple), portanto, consulte-o se tiver dúvidas sobre os tópicos abordados aqui.

Observe que, diferentemente do último laboratório, você trabalhará com o TF servindo sem usar o Docker. O objetivo é mostrar a você as diferentes maneiras de usar esse sistema de serviço.

Vamos começar!

### Importações

In [ ]:
import os
import zipfile
import subprocess
import numpy as np
import pandas as pd
import tensorflow as tf
from IPython.display import Image, display

### Baixando os dados

Neste laboratório, você não treinará um modelo, mas usará um modelo para obter previsões, por isso precisa de algumas imagens de teste. O modelo que você usará foi originalmente treinado com imagens dos conjuntos de dados *cats and dogs* e  *Caltech birds*. Para solicitar previsões, são fornecidas algumas imagens do conjunto de teste:

In [ ]:
# Baixar as imagens
!wget -q https://storage.googleapis.com/mlep-public/course_3/week2/images.zip

# Definir o diretório base
base_dir = '/tmp/data'

# Descompactar as imagens
with zipfile.ZipFile('/content/images.zip', 'r') as my_zip:
  my_zip.extractall(base_dir)

# Salvar os caminhos para as imagens de cada classe
dogs_dir = os.path.join(base_dir, 'images/dogs')
cats_dir = os.path.join(base_dir,'images/cats')
birds_dir = os.path.join(base_dir,'images/birds')

# Imprimir a quantidade de imagens de cada classe
print(f"Há {len(os.listdir(dogs_dir))} imagens de cachorros")
print(f"Há {len(os.listdir(cats_dir))} imagens de gatos")
print(f"Há {len(os.listdir(birds_dir))} imagens de pássaros\n\n")

# Veja exemplos de imagens de cada classe
print("Exemplo de imagem de gato:")
display(Image(filename=f"{os.path.join(cats_dir, os.listdir(cats_dir)[0])}"))
print("\nExemplo de imagen de cachorro:")
display(Image(filename=f"{os.path.join(dogs_dir, os.listdir(dogs_dir)[0])}"))
print("\nExemplo de imagem de pássaro:")
display(Image(filename=f"{os.path.join(birds_dir, os.listdir(birds_dir)[0])}"))

Agora que você está familiarizado com os dados com os quais trabalhará, vamos passar para o modelo.

### Carregar um modelo pré-treinado

O objetivo deste laboratório é demonstrar os recursos do TF serving, portanto, você não investirá tempo treinando um modelo. Em vez disso, você usará um modelo pré-treinado. Esse modelo classifica imagens de pássaros, gatos e cachorros e foi treinado com aumento de imagem para produzir resultados muito bons.

Primeiro, faça o download dos arquivos necessários:

In [ ]:
!wget -q -P /content/model/ https://storage.googleapis.com/mlep-public/course_1/week2/model-augmented/saved_model.pb
!wget -q -P /content/model/variables/ https://storage.googleapis.com/mlep-public/course_1/week2/model-augmented/variables/variables.data-00000-of-00001
!wget -q -P /content/model/variables/ https://storage.googleapis.com/mlep-public/course_1/week2/model-augmented/variables/variables.index

Agora, carregue o modelo na memória:

In [ ]:
model = tf.keras.models.load_model('/content/model')

Neste ponto, você pode presumir que treinou o modelo com sucesso. Você pode ignorar os avisos sobre o modelo ter sido treinado em uma versão mais antiga do TensorFlow.

Para fins de contexto, esse modelo usa uma arquitetura CNN simples. Dê uma olhada rápida nas camadas que o compõem:

In [ ]:
model.summary()

## Salve seu modelo

Para carregar nosso modelo treinado no TensorFlow Serving, primeiro precisamos salvá-lo no formato [SavedModel](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/saved_model).

Isso criará um arquivo [protobuf](https://developers.google.com/protocol-buffers) em uma hierarquia de diretórios bem definida e incluirá um número de versão.

O [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving) nos permite selecionar qual versão de um modelo, ou "servível", queremos usar quando fizermos solicitações de inferência.

Cada versão será exportada para um subdiretório diferente no caminho fornecido.

In [ ]:
# Buscar a sessão do Keras e salvar o modelo
# A definição da assinatura é definida pelos tensores de entrada e saída,
# e armazenada com a chave de serviço padrão
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print(f'export_path = {export_path}\n')


# Salvar o modelo
tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

Um modelo salvo em disco inclui os seguintes arquivos:
- `assets`: um diretório que inclui arquivos arbitrários usados pelo grafo TF.
- `variables`: um diretório que contém informações sobre os pontos de verificação de treinamento do modelo.
- `saved_model.pb`: o arquivo protobuf que representa o programa TF real.

Dê uma olhada rápida nesses arquivos:

In [ ]:
print(f'\nArquivos do modelo salvos em {export_path }:\n')
!ls -lh {export_path}

## Examine seu modelo salvo

Usaremos o utilitário de linha de comando `saved_model_cli` para examinar os [MetaGraphDefs](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/MetaGraphDef) (os modelos) e [SignatureDefs](https://www.tensorflow.org/tfx/serving/signature_defs) (os métodos que você pode chamar) em nosso SavedModel.

Para maiores detalhes consulte [essa discussão sobre a CLI do SavedModel](https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/saved_model.md#cli-to-inspect-and-execute-savedmodel) no Guia do TensorFlow.

In [ ]:
!saved_model_cli show --dir {export_path} --tag_set serve --signature_def serving_default

Isso nos diz muito sobre o nosso modelo!

Nesse caso, não treinamos explicitamente o modelo, portanto, qualquer informação sobre as entradas e saídas é muito valiosa. 

Por exemplo, sabemos que esse modelo espera que nossas entradas tenham a forma `(150, 150, 3)`, o que, combinado com o uso de camadas `conv2d`, sugere que esse modelo espera imagens coloridas em uma resolução de `150 por 150`.

Além disso, a saída do modelo tem a forma `(3)`, o que sugere uma ativação `softmax` com 3 classes.

## Preparar dados para inferência

Agora que você conhece a forma dos dados esperados pelo modelo, é hora de pré-processar as imagens de teste adequadamente. 

Essas imagens são fornecidas em uma ampla variedade de resoluções, e felizmente o Keras tem o que você precisa com seu [`ImageDataGenerator`](https://keras.io/api/preprocessing/image/).

Usando esse objeto, você pode:
- Normalizar valores de pixel.
- Padronizar resoluções de imagem.
- Definir um tamanho de lote para inferência.
- E muito mais!

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Normalizar os valores dos pixels
test_datagen = ImageDataGenerator(rescale=1./255)

# Apontar para o diretório com as imagens de teste
val_gen_no_shuffle = test_datagen.flow_from_directory(
    '/tmp/data/images',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=True)

# Imprimir o rótulo associado a cada classe
print(f"Os rótulos de cada classe no gerador de testes são: {val_gen_no_shuffle.class_indices}")

Como esse objeto é um [gerador](https://wiki.python.org/moin/Generators), você pode obter um lote de imagens e rótulos usando a função `next`:

In [ ]:
# Obtenha um lote de 32 imagens junto com seu rótulo verdadeiro
data_imgs, labels = next(val_gen_no_shuffle)

# Verifique os formatos
print(f"data_imgs tem formato: {data_imgs.shape}")
print(f"os rótulos tem o formato: {labels.shape}")

As expected `data_imgs` is an array containing 32 colored images of 150x150 resolution. In a similar fashion `labels` has the true label for each one of these 32 images.

To check that everything is working properly, do a sanity check to plot the first 5 images in the batch:

In [ ]:
from tensorflow.keras.preprocessing.image import array_to_img

# Returns string representation of each class
def get_class(index):
  if index == 0:
    return "bird"
  elif index == 1:
    return "cat"
  elif index == 2:
    return "dog"
  return None


# Plots a numpy array representing an image
def plot_array(array, label, pred=None):
  array = np.squeeze(array)
  img = array_to_img(array)
  display(img)
  if pred is None:
    print(f"Image shows a {get_class(label)}.\n")
  else:
    print(f"Image shows a {get_class(label)}. Model predicted it was {get_class(pred)}.\n")


# Plot the first 5 images in the batch
for i in range(5):
  plot_array(data_imgs[i], labels[i])

All images have the same resolution and the true labels are correct.

Let's jump to serving the model!

## Serve your model with TensorFlow Serving


### Install TensorFlow Serving

You will need to install an older version (2.8.0) because more recent versions are currently incompatible with Colab.

In [ ]:
!wget 'http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-universal-2.8.0/t/tensorflow-model-server-universal/tensorflow-model-server-universal_2.8.0_all.deb'
!dpkg -i tensorflow-model-server-universal_2.8.0_all.deb

### Start running TensorFlow Serving

This is where we start running TensorFlow Serving and load our model.  After it loads we can start making inference requests using REST.  There are some important parameters:

* `rest_api_port`: The port that you'll use for REST requests.
* `model_name`: You'll use this in the URL of REST requests.  It can be anything. For this case `animal_classifier` is used.
* `model_base_path`: This is the path to the directory where you've saved your model.


In [ ]:
# Define an env variable with the path to where the model is saved
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
# Spin up TF serving server
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=animal_classifier \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Take a look at the end of the logs printed out my TF model server:

In [ ]:
!tail server.log

The server was able to succesfully load and serve the model!

Since you are going to interact with the server through HTTP/REST, you should point the requests to `localhost:8501` as it is being printed in the logs above.

## Make a request to your model in TensorFlow Serving

At this point you already know how your test data looks like. You are going to make predictions for colored images of 150x150 in batches of 32 images (represented by numpy arrays) at a time.

Since REST expects the data to be in JSON format and JSON does not support custom Python data types such as numpy arrays you first need to convert these arrays into nested lists.

TF serving expects a field called `instances` which contains the input tensors for the model. To pass in your data to the model you should create a JSON with your data as value for the key `instances`.

In [ ]:
import json

# Convert numpy array to list
data_imgs_list = data_imgs.tolist()

# Create JSON to use in the request
data = json.dumps({"instances": data_imgs_list})

### Make REST requests

We'll send a predict request as a POST request to our server's REST endpoint, and pass it the batch of 32 images. 

Remember that the endpoint that serves the model is located at `http://localhost:8501`. However this URL still needs some additional parameters to properly handle the request. You should append `v1/models/name-of-your-model:predict` to it so TF serving knows which model to look for and to perform a predict task.

You should also pass to the request the data containing the list that represents the 32 images along with a headers dictionary that specifies the type of content that will be passed, which is JSON in this case.

After you get a response from the server you can get the predictions out of it by inspecting the `predictions` field of the JSON that the response returned.

In [ ]:
import requests

# Define headers with content-type set to json
headers = {"content-type": "application/json"}

# Capture the response by making a request to the appropiate URL with the appropiate parameters
json_response = requests.post('http://localhost:8501/v1/models/animal_classifier:predict', data=data, headers=headers)

# Parse the predictions out of the response
predictions = json.loads(json_response.text)['predictions']

# Print shape of predictions
print(f"predictions has shape: {np.asarray(predictions).shape}")

You might think it is weird that the predictions returned 3 values for each image. However, remember that the last layer of the model is a `softmax` function, so it returned a value for each one of the class. To get the actual predictions you need to find the maximum argument:

In [ ]:
# Compute argmax
preds = np.argmax(predictions, axis=1)

# Print shape of predictions
print(f"preds has shape: {preds.shape}")

Now you have a predicted class for each one of the test images! Nice!

To test how good the model is performing let's plot the first 10 images along with the true and predicted labels:

In [ ]:
for i in range(10):
  plot_array(data_imgs[i], labels[i], preds[i])

To do some further testing you can plot more images out of the 32 or even try to generate a new batch from the generator and repeat the steps above.

## Optional Challenge

Try to recreating the steps above for the next batch of 32 images:

In [ ]:
# Your code here



## Solution

If you want some help, the answer can be found in the next cell:

In [ ]:
# Get a batch of 32 images along with their true label
data_imgs, labels = next(val_gen_no_shuffle)

# Convert numpy array to list
data_imgs_list = data_imgs.tolist()

# Create JSON to use in the request
data = json.dumps({"instances": data_imgs_list})

# Capture the response by making a request to the appropiate URL with the appropiate parameters
json_response = requests.post('http://localhost:8501/v1/models/animal_classifier:predict', data=data, headers=headers)

# Parse the predictions out of the response
predictions = json.loads(json_response.text)['predictions']

# Compute argmax
preds = np.argmax(predictions, axis=1)

for i in range(5):
  plot_array(data_imgs[i], labels[i], preds[i])

# Conclusion
**Congratulations on finishing this ungraded lab!**

Now you should have a deeper understanding of TF serving's internals. In the previous ungraded lab you saw how to use TFS alongside with Docker. In this one you saw how TFS and `tensorflow-model-server` worked on their own. You also saw how to save a model and the structure of a saved model.

**Keep it up!**